In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI

In [3]:
# Load API Credentials
with open('/Users/alicia/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# set our API call parameters and filename before the first call
LOCATION = 'Baltimore, MD,21202'
TERM = 'Crab Cakes'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Crab_cakes.json"
JSON_FILE

'Data/results_in_progress_Crab_cakes.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Crab_cakes.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

436

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

22

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/22 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/22 [00:00<?, ?it/s]

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,D9A33FM394q99o4QtK5YwA,faidleys-seafood-baltimore-3,Faidley's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/8j3ynZ...,False,https://www.yelp.com/biz/faidleys-seafood-balt...,1199,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.291696, 'longitude': -76.62224}",[delivery],$$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107274898,(410) 727-4898,1349.560720
1,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media1.fl.yelpcdn.com/bphoto/9hGjo5...,False,https://www.yelp.com/biz/thames-street-oyster-...,2762,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,2090.712792
2,u65W69AhbjUlvJJBkEhGNQ,miss-shirleys-cafe-baltimore-9,Miss Shirley's Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/9FsOyV...,False,https://www.yelp.com/biz/miss-shirleys-cafe-ba...,2977,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 39.2870995, 'longitude': -76.6053...",[delivery],$$,"{'address1': '750 E Pratt St', 'address2': '',...",+14105285373,(410) 528-5373,1028.736468
3,1C8QSqc-erkkyAUywh501A,angies-seafood-baltimore,Angie's Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/pqkYsm...,False,https://www.yelp.com/biz/angies-seafood-baltim...,707,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.289207148922245, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '1727 E Pratt St', 'address2': No...",+14103420917,(410) 342-0917,1499.083404
4,6am8TZAFnvND52MOz-Yctg,mamas-on-the-half-shell-baltimore,Mama's On The Half Shell,https://s3-media2.fl.yelpcdn.com/bphoto/HWY8OF...,False,https://www.yelp.com/biz/mamas-on-the-half-she...,1294,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.27986, 'longitude': -76.5752399}","[pickup, delivery]",$$,"{'address1': '2901 Odonnell St', 'address2': '...",+14102763160,(410) 276-3160,3328.825798


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
423,WVQqkLAKowuIe51IVNXCZA,outback-steakhouse-baltimore,Outback Steakhouse,https://s3-media2.fl.yelpcdn.com/bphoto/NzwBx9...,False,https://www.yelp.com/biz/outback-steakhouse-ba...,136,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3.0,"{'latitude': 39.2817468, 'longitude': -76.5816...","[delivery, pickup]",$$,"{'address1': '2400 Boston St', 'address2': '',...",+14105227757,(410) 522-7757,2755.986468
424,dWI0jCoOqjge9VQTIad-Ww,york-pizza-subs-and-wings-baltimore,York Pizza Subs and Wings,https://s3-media4.fl.yelpcdn.com/bphoto/Ay3jyi...,False,https://www.yelp.com/biz/york-pizza-subs-and-w...,51,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"{'latitude': 39.3298530578613, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '3401 Greenmount Ave', 'address2'...",+14102356290,(410) 235-6290,3743.767034
425,nA8hY_b-yD4U_PQGWGmw9Q,american-wings-and-pizza-baltimore,American Wings & Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/mYgJ5p...,False,https://www.yelp.com/biz/american-wings-and-pi...,61,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 39.3165, 'longitude': -76.61556}","[delivery, pickup]",$,"{'address1': '2400 St Paul St', 'address2': ''...",+14102352300,(410) 235-2300,2361.614576
426,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.33086, 'longitude': -76.63161}","[delivery, pickup]",$$,"{'address1': '907 W 36th St', 'address2': None...",+14108893663,(410) 889-3663,4382.210159
427,EWkA_yQ1VPa7cHbkiBbawA,phillys-best-baltimore,Philly's Best,https://s3-media4.fl.yelpcdn.com/bphoto/CXQdUz...,False,https://www.yelp.com/biz/phillys-best-baltimor...,45,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 39.33084, 'longitude': -76.63393}","[delivery, pickup]",$,"{'address1': '1101 W 36th St', 'address2': '',...",+14102431301,(410) 243-1301,4467.724059


In [16]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

0

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip',index=False)